## Reference Transcriptome for Three _Abies_
xanadu path: `/labs/Wegrzyn/Transcriptomics/xmas_trees/NAR/poola/transcriptomes`
github: `https://github.com/alextrouerntrend/fir-transcriptomics`

**Tools:**
* [Trinity (v 2.2.0)](https://github.com/trinityrnaseq/trinityrnaseq/wiki)
* [GeneMarkS-T](http://topaz.gatech.edu/GeneMark/)
* [VSEARCH (v 1.1.1)](https://github.com/torognes/vsearch)
* [EnTAP](https://entap.readthedocs.io/en/latest/)
* [OrthoFinder (v 1.1.8)](https://davidemms.github.io/)

**Transcriptomes:**
* Fraser fir (_Abies fraseri_) 
* balsam fir (_Abies balsamea_)
* Canaan fir (_Abies balsamea_ var. phanerolepis)

### Reference Transcriptomes for each species
1. A _de novo_ assembly for each library was constructed using [Trinity (v 2.2.0)](https://github.com/trinityrnaseq/trinityrnaseq/wiki). 
2. Frame selection of assembled contigs was executed using [GeneMarkS-T](http://topaz.gatech.edu/GeneMark/), a tool for ab initio identification of the protein coding regions in RNA transcripts. 
3. Individually, for each species, assembled transcripts from all libraries were concatenated into a single file. 
4. The contigs in the merged file were clustered at .90 identity using [VSEARCH (v 1.1.1)](https://github.com/torognes/vsearch)
5. Functional annotation via [EnTAP](https://entap.readthedocs.io/en/latest/) (see methods below) identified sequences that aligned with confidence to fungal, bacterial, amoeba, and insect full length proteins in NCBI’s RefSeq database. These contaminant transcripts were removed from the data.

### Combined Reference Strategies
Two strategies were employed to build reference transcriptomes which included transcripts from the three taxa. The first strategy clustered the three single species transciptomes together using VSEARCH. The second strategy carries out the same VSEARCH clustering, but does so after phylogenetic orthology inference with OrthoFinder. In the second strategy, each resulting orthogroup is input into VSEARCH separately, resulting in 61400 and 86989 separate VSEARCH runs, depending on the version of OrthoFinder, one for each orthogroup. In both cases the centroids from all resulting VSEARCH clusters were taken to build reference transcriptomes. Experimental filters were applied to enable the resolution of patterns in gene expression that are common across the three taxa.

#### Orthofinder Strategy
Using [OrthoFinder](https://davidemms.github.io/), frame-selected transcripts from the three reference transcriptomes were clustered into orthologous gene families. click filenames below to view statistics from that run. Xanadu path: `/labs/Wegrzyn/Transcriptomics/xmas_trees/NAR/poola/transcriptomes/vsearch_splitting` Two versions of Orthofinder were tested: (v 1.1.8), (v 2.4.0)

The primary distinction between the two OrthoFinder runs is that the number of species-specific orthogroup formations greatly increases when using OrthoFinder2. This causes an increase in the total number of orthogroups.
[Overall Stats Comparison](https://github.com/alextrouerntrend/fir-transcriptomics/blob/master/img/of_stat_overall.png")



**OrthoFinder1 Stats**
[Statistics_Overall](https://github.com/alextrouerntrend/fir-transcriptomics/blob/master/data/orthofinder/of1/Statistics_Overall.csv)
[Statistics_PerSpecies](https://github.com/alextrouerntrend/fir-transcriptomics/blob/master/data/orthofinder/of1/Statistics_PerSpecies.csv)

**OrthoFinder2 Stats**
[Statistics_Overall](https://github.com/alextrouerntrend/fir-transcriptomics/blob/master/data/orthofinder/of2/Statistics_Overall.tsv)
[Statistics_PerSpecies](https://github.com/alextrouerntrend/fir-transcriptomics/blob/master/data/orthofinder/of2/Statistics_PerSpecies.tsv)

To experiment with optimal clustering identity, orthogroups were then clustered at 90% and 85% using VSEARCH to split  divergent transcripts into independent clusters. For each clustering identity, centroids from every cluster were selected to consistute the combined reference transcriptome.

##### **Orthofinder Methods**
```bash
# Orthofinder.csv was split into 13 files of 5000 lines for parallel processing.
split Orthogroups.csv -l 5000 Orthogroups
```
Submit 13 scripts to run of the split files.
```bash
for x in {a..m}; do if [[ -v PRE ]]; then sed -i "s/Orthogroupsa$PRE/Orthogroupsa$x/g" vsearch_single.sh && sbatch vsearch_single.sh && PRE=$x; else sbatch vsearch_single.sh && PRE=$x; fi; done
```

[vsearch_single.sh](https://github.com/alextrouerntrend/fir-transcriptomics/blob/master/scripts/vsearch_single.sh), which calls [grabseqs.py](https://github.com/alextrouerntrend/fir-transcriptomics/blob/master/scripts/grabseqs.py), was run on files of 5000 orthogroups in parallel.

After vsearch runs finished, the transcriptome was created by concatenating the centroid files from every run.
```bash
find . -name "*_centroids.fnn" -print0 | xargs -0 cat > fir90_centroids.fnn
```

### VSEARCH Strategy
Reference transcriptomes for the three taxa were concatenated into a single file and experimentally clustered using VSEARCH at 85% and 90%. For each clustering identity, centroids from every cluster were selected to consistute the combined reference transcriptome.

<details>
    <summary> <b>Sanity Check</b> (click) - do # of seqs in constructed .fnn files match number in OG? <b> Yes, they do</b></summary>

```
atrend@xanadu-02 /labs/Wegrzyn/Transcriptomics/xmas_trees/NAR/poola/transcriptomes/vsearch_splitting/output2
$ grep -c ">" */*.fnn | grep -v "centroids"
id_OG0000000/OG0000000.fnn:107
id_OG0000001/OG0000001.fnn:59
id_OG0000002/OG0000002.fnn:54
id_OG0000003/OG0000003.fnn:53
id_OG0000004/OG0000004.fnn:51
id_OG0000005/OG0000005.fnn:45
id_OG0000006/OG0000006.fnn:41
id_OG0000007/OG0000007.fnn:41
id_OG0000008/OG0000008.fnn:39

atrend@xanadu-02 /labs/Wegrzyn/Transcriptomics/xmas_trees/NAR/poola/transcriptomes/vsearch_splitting
$ grep -o -n 'CF\|Fraser\|Balsam' Orthogroups.csv |  cut -d : -f 1 | uniq -c | head -n 10
      1 1
    107 2
     59 3
     54 4
     53 5
     51 6
     45 7
     41 8
     41 9
     39 10

```

</details>

### Filtering
Clustering statistics were calculated using the custom python script, [clusterStats.py](https://github.com/alextrouerntrend/fir-transcriptomics/blob/master/scripts/clustStats.py), which searches the directories created by VSEARCH clustering by [vsearch_single.sh](https://github.com/alextrouerntrend/fir-transcriptomics/blob/master/scripts/vsearch_single.sh) and creates a stats table like [o85.tsv](https://github.com/alextrouerntrend/fir-transcriptomics/blob/master/data/transcriptome_stats/o85.tsv).

The tables were processed in RStudio using [transcStats.R](https://github.com/alextrouerntrend/fir-transcriptomics/blob/master/scripts/transcStats.R), which applied two cluster membership filters to determine cluster retention.
1. Cluster must include >1 species.
2. Cluster must include fraser fir transcript.

After filtering, centroids, filtered statistics tables and a summary table were exported from RStudio. These are available in github.
1. [centroids](https://github.com/alextrouerntrend/fir-transcriptomics/tree/master/data/centroids).
2. [stats tables](https://github.com/alextrouerntrend/fir-transcriptomics/tree/master/data/transcriptome_stats)
3. [summary table](https://github.com/alextrouerntrend/fir-transcriptomics/tree/master/data/transcriptome_stats/tn_summary.csv)
4. [orthofinder results](https://github.com/alextrouerntrend/fir-transcriptomics/tree/master/data/orthofinder): includes stats, orthogroup.tsv, unassigned genes and single-copy orthogroups.

### Read mapping to transcriptomes
Trimmed reads from libraries of the three fir taxa were mapped to the Orthofinder2-derived reference transcriptomes using two implementations of the bwa-mem algorithm, [BWA-mem](http://bio-bwa.sourceforge.net/bwa.shtml) and an updated version, [BWA-mem2](https://github.com/bwa-mem2/bwa-mem2). The two versions of the algorithm were benchmarked against one another for speed and alignment rate. Jobs were submitted as array.

file location: `/labs/Wegrzyn/Transcriptomics/xmas_trees/NAR/poola/analyses/mapping`

Scripts:
1. [bwa-index.sh](https://github.com/alextrouerntrend/fir-transcriptomics/blob/master/scripts/bwa-index.sh)
2. [bwa-map.sh](https://github.com/alextrouerntrend/fir-transcriptomics/blob/master/scripts/bwa-map.sh)